# Rappel sur les enregistrements


On rappelle qu’on peut définir en `OCaml` des types proches des `struct`
du `C`. On parle de **type enregistrement** (ou *record* en anglais) en
référence à un parallèle avec une fiche comme celles que l’on vous
demande de remplir pour les inscriptions : un certain nombre de *champs*
ayant une référence précise comme *adresse* ou *numéro de téléphone* et
un type de valeur attendu pour les réponses.

En `OCaml`, pour stocker le nom, l’age et la moyenne d’informatique d’un
élève, on pourrait ainsi imaginer un enregistrement à trois champs et le
définir ainsi :

``` ocaml
type eleve = {
    nom : string;
    age : int;
    moyenne : float
}
```

L’ordre dans lequel les champs sont énumérés n’importe pas. Pour créer
une valeur du type `eleve`, il suffit de renseigner les champs, dans
l’ordre qu’on veut, comme si on remplissait une fiche
`{ nom = "Emile"; age = 18; moyenne = 19.9 }`.

Si on dispose d’une valeur `e` du type `eleve`, on accède au valeur de
ses champs en utilisant `.` comme en `C` : `e.nom` pour avoir son nom.

Les valeurs des champs sont **immuables** par défaut, cela signifie
qu’on ne peut pas changer la moyenne de l’élève ! Comme souvent,
travailler avec des données persistantes est plus simples et ce choix
par défaut est raisonnable, mais dans de nombreux cas, on aimerait que
certaines valeurs puissent changer. À cette fin, on peut déclarer un
champ `mutable`. On va ainsi redéfinir `eleve` ainsi :

In [ ]:
type eleve = {
    nom : string;
    age : int;
    mutable moyenne : float
}

On aurait alors à écrire `e.moyenne <- 12.4` pour changer la valeur de
la moyenne.

**Attention** il est interdit d’écrire cela si le champ n’est pas
`mutable`.

On rappelle que les références de `OCaml` sont en fait des types
enregistrement `'a ref` avec un unique champ `mutable contents : 'a`. On
a ainsi une correspondance parfaite entre les notations :

-   `ref x` est en fait `{ contents = x }`
-   l’évaluation `!x` est `x.contents`
-   `x := v` est en fait `x := v`

**Question 1**

Écrire une fonction `plus_age : eleves list -> eleve` qui prend en
entrée une liste d’élèves **non vide** et renvoie l’élève le plus agé.

In [ ]:
let rec plus_age l =
match l with
| [] -> {nom = "Dieu"; age = -1; moyenne = 42.}
| [e]-> e
| t::q-> let other = plus_age q in 
if other.age > t.age then other  else t

In [ ]:
let _ =
    let e1 = { nom = "A"; age = 12; moyenne = 10.0 } in
    let e2 = { nom = "A"; age = 15; moyenne = 9.0 } in
    let e3 = { nom = "A"; age = 13; moyenne = 11.0 } in
    let classe = [ e1; e2; e3 ] in
    assert(plus_age classe = e2);
    "OK"


**Question 2**

Écrire une fonction `moyenne : eleves list -> float` qui prend en entrée
une liste d’élèves **non vide** et renvoie la moyenne des élèves.

**Rappel** les flottants en `OCaml` sont un peu pénibles car ils ont des
opérateurs dédiés comme `+.` et `/.`. Pour convertir un `int` en `float`
on peut utiliser `float_of_int : int -> float`.

In [ ]:
let moyenne l =
let rec aux l = 
match l with
| [] -> 42.,0.
| [e]-> e.moyenne,1.
| t::q-> let a,b = aux q in a+.t.moyenne,b+.1. in
let s,n = aux l in s/.n

In [ ]:
let _ =
    let e1 = { nom = "A"; age = 12; moyenne = 10.0 } in
    let e2 = { nom = "A"; age = 15; moyenne = 9.0 } in
    let e3 = { nom = "A"; age = 13; moyenne = 11.0 } in
    let classe = [ e1; e2; e3 ] in
    assert(moyenne classe = 10.0);
    "OK"


**Question 3**

Écrire une fonction `bulletin : eleves list -> unit` qui rajoute 2
points à la moyenne de tous les élèves de la liste.

In [ ]:
let rec bulletin l = 
    match l with
    |[] -> ()
    | e::q -> e.moyenne <- e.moyenne +. 2.; bulletin q


In [ ]:
let _ =
    let e1 = { nom = "A"; age = 12; moyenne = 10.0 } in
    let e2 = { nom = "A"; age = 15; moyenne = 9.0 } in
    let e3 = { nom = "A"; age = 13; moyenne = 11.0 } in
    let classe = [ e1; e2; e3 ] in
    ignore (bulletin classe);
    assert(e1.moyenne = 12.0);
    assert(e2.moyenne = 11.0);
    assert(e3.moyenne = 13.0);
    "OK"


# Références et pointeurs

On va se servir des références pour faire des pointeurs comme en `C`.

On définit le type

In [1]:
type 'a ptr = 'a option ref

type 'a ptr = 'a option ref

**Note** on a ici à la fois une référence et un type option, tout
l’enjeu de l’exercice est d’apprendre à bien manipuler ce type qu’on
retrouvera dans beaucoup de contextes où on a besoin d’une information
partielle, comme lorsqu’on veut résoudre petit à petit une grille de
*Sudoku*.

On rappelle que le type `'a option` est prédéfini et qu’il correspond au
type somme `type 'a option = None | Some of 'a`. La valeur `None`
signifie qu’il n’y a pas d’information et `Some v` signifie qu’on a une
information de valeur `v`.

Pour utiliser une valeur de type option il est ainsi nécessaire
d’effectuer un filtrage :

``` ocaml
match x with
| None -> (* ce qu'on fait quand il n'y a rien *)
| Some v -> (* ce qu'on fait quand il y a une valeur v *)
```

Pour plus d’information sur le type `option`, on pourra lire ce
[chapitre du
poly](https://poly.mpi.guru/chap_programmation_options_ocaml.html).

Ici, un pointeur sur la valeur `v` sera donc défini par `ref (Some v)`.
Le pointeur invalide que l’on note `NULL` en `C` sera défini par
`ref None`.

**Question 4**

Écrire une fonction `null_pointer : unit -> 'a ptr` qui renvoie le
pointeur nul.

In [2]:
let null_pointer () = ref None

val null_pointer : unit -> 'a option ref = <fun>

In [3]:
let _ =
    let p = null_pointer () in
    assert(!p = None);
    "OK"


- : string = "OK"

**Question 5**

Écrire une fonction `is_null : 'a ptr -> bool` qui renvoie un booléen
indiquant si un pointeur est nul.

In [4]:
let is_null p = !p = None

val is_null : 'a option ref -> bool = <fun>

In [5]:
let _ =
    assert(is_null (ref None));
    assert(not (is_null (ref (Some 3))));
    "OK"


- : string = "OK"

**Question 6**

Écrire une fonction `dereference : 'a ptr -> 'a` qui renvoie la valeur
pointée par un pointeur.

Cette fonction devra produire une erreur avec un `failwith` si le
pointeur est nul.

In [6]:
let dereference p = 
    match !p with None -> failwith "caca" | Some v -> v

val dereference : 'a option ref -> 'a = <fun>

In [7]:
let _ =
    let p = ref (Some 3) in
    assert (dereference p = 3);
    "OK"


- : string = "OK"

**Question 7**

Écrire une fonction `set_pointer : 'a ptr -> 'a -> unit` qui change, ou
définit s’il était nul, la valeur pointée par un pointeur.

In [8]:
let set_pointer p v = (p := Some v)

val set_pointer : 'a option ref -> 'a -> unit = <fun>

In [9]:
let _ =
    let p = ref None in
    set_pointer p 3;
    assert(p = ref (Some 3));
    set_pointer p 5;
    assert(p = ref (Some 5));
    "OK"


- : string = "OK"

Ce n’est pas très pratique d’écrire `dereference p` ou
`set_pointer p v`. On peut utiliser des raccourcis avec des opérateurs
préfixes et infixes. Ainsi, on peut définir :

In [10]:
let (!) p = dereference p
let (<==) p v = set_pointer p v

val ( ! ) : 'a option ref -> 'a = <fun>
val ( <== ) : 'a option ref -> 'a -> unit = <fun>

On peut alors directement utiliser ces symboles ainsi :

In [11]:
let p = null_pointer () in
p <== 3; (* p pointe vers 3 *)
!p (* la valeur pointée, donc 3 *)

- : int = 3

Pour le moment, l’usage semble un peu limité, on va voir une utilisation
possible : initialiser des paramètres dans des boucles.

In [12]:
let recherche t x = (* renvoie l'indice de x dans t s'il est présent *)
    let p = null_pointer () in
    for i = 0 to Array.length t - 1 do
        if t.(i) = x
        then p <== i
    done;
    !p

val recherche : 'a array -> 'a -> int = <fun>

# A link to the future

*Cette partie prend un peu d’avance sur le cours sur les listes
chaînées.*

On va considérer maintenant le type

In [14]:
type link = {
    mutable value : int;    
    next : link ptr
}

type link = { mutable value : int; next : link ptr; }

Les valeurs de ce type seront appelées des **maillons** et leur
assemblage constitue une **chaîne**.

Un maillon de type `link` contient ainsi une valeur, qui peut changer,
et un pointeur vers le maillon suivant. Quand il n’y a pas de maillon
qui fait suite, on placera le pointeur nul.

En suivant les pointeurs `suivant`, on voit ainsi apparaitre une chaine
:

    <3,|>       |---- <1, NUL>
       |---> <2,|>

Cette chaîne est ainsi une autre manière de représenter une liste comme
`[1;2;3]`.

Dans les questions suivantes, on utilise le mot *chaine* quand il s’agit
de voir non seulement un maillon, mais l’ensemble de la chaine qui s’en
déduit.

**Question 8**

Écrire une fonction `single_link : int -> link` qui crée une chaine
comportant un unique maillon contenant une valeur.

In [15]:
let single_link x =  {value = x; next = ref None}

val single_link : int -> link = <fun>

In [16]:
let _ =
    let l = single_link 3 in
    assert(l = { value = 3; next = ref None });
    "OK"


- : string = "OK"

**Question 9**

Écrire une fonction `link_connect : link -> link -> unit` telle que
`link m1 m2` place `m2` comme maillon suivant de `m1`.

In [30]:
let link_connect m1 m2 = m1.next <== m2

val link_connect : link -> link -> unit = <fun>

In [31]:
let _ =
    let m1 = { value = 3; next = ref None } in
    let m2 = { value = 1; next = ref None } in
    link_connect m1 m2;
    assert(!(m1.next) = m2);
    "OK"


- : string = "OK"

**Question 10**

Écrire une fonction `is_last : link -> bool` qui renvoie un booléen
indiquant si un maillon est le dernier.

**Remarque** : il faut utiliser `is_null`.

In [32]:
let is_last m = is_null m.next


val is_last : link -> bool = <fun>

In [33]:
let _ =
    let m = { value = 3; next = ref None } in
    let m' = { value = 3; next = ref (Some m) } in
    assert(is_last m);
    assert(not (is_last m'));
    "OK"


- : string = "OK"

**Question 11**

Écrire une fonction `next_link : link -> link` qui renvoie le maillon
suivant après un maillon qui n’est pas le dernier.

**Remarque** : il faut utiliser `dereference` ou `!`.

In [35]:
let next_link m = dereference m.next

val next_link : link -> link = <fun>

In [36]:
let _ =
    let m1 = { value = 3; next = ref None } in
    let m2 = { value = 1; next = ref None } in
    link_connect m1 m2;
    assert(next_link m1 = m2);
    "OK"


- : string = "OK"

**Question 12**

Écrire une fonction `link_of_list : int list -> link` qui crée une
chaîne depuis une liste d’entiers **non vide**.

In [49]:
let rec link_of_list l = 
    match l with
    | [] -> failwith "on a dit non vide vindieu"
    | [t] -> single_link t
    | t::q -> let c = single_link t in link_connect c (link_of_list q);c


val link_of_list : int list -> link = <fun>

In [51]:
let _ =
    let s = link_of_list [1;2;3] in
    
    let m3 = { value = 3; next = ref None } in
    let m2 = { value = 2; next = ref (Some m3) } in
    let m1 = { value = 1; next = ref (Some m2) } in
    assert(s = m1);
    "OK"

- : string = "OK"

Dans les fonctions suivantes, on fera les parcours de chaine uniquement
en récursif mais sans `match` en utilisant `is_last` et `next_link` à la
place.

**Question 13**

Écrire une fonction `last_link : link -> link` qui renvoie le dernier
maillon d’une chaîne en descendant, tant que c’est possible, le long des
pointeurs `suivant`.

In [54]:
let rec last_link m =
    if is_last m then m else last_link (next_link m)


val last_link : link -> link = <fun>

In [55]:
let _ =
    let l = link_of_list [1;2;3] in
    assert(last_link l = { value = 3; next = ref None });
    "OK"


- : string = "OK"

**Question 14**

Écrire une fonction `print_link : link -> unit` qui affiche les éléments
d’une chaîne.

In [61]:
let rec print_link m =
    begin
        print_int m.value;
        print_char ' ';
        if (is_last m)=false then print_link (next_link m) 
    end

val print_link : link -> unit = <fun>

In [62]:
let _ =
    let l = link_of_list [1;2;3] in
    print_link l


1 2 3 

- : unit = ()

**Question 15**

Écrire une fonction `link_to_list : link -> int list` qui renvoie la
liste des valeurs dans une chaîne.

In [63]:
let rec link_to_list m = 
    if is_last m then [m.value] else m.value::link_to_list (next_link m)

val link_to_list : link -> int list = <fun>

In [64]:
let _ =
    let l = link_of_list [3;2;1] in
    let ll = link_to_list l in
    assert(ll = [3;2;1]);
    "OK"


- : string = "OK"

**Question 16**

En déduire une fonction `append : link -> link -> unit` qui colle la
seconde chaîne à la fin de la première.

In [66]:
let append m1 m2 =
    let rec last m = if is_last m then m else last (next_link m)
    in link_connect (last m1) m2

val append : link -> link -> unit = <fun>

In [67]:
let _ =
    let m1 = link_of_list [1;2] in
    let m2 = link_of_list [3;4] in
    append m1 m2;
    assert(link_to_list m1 = [1;2;3;4]);
    "OK"


- : string = "OK"